In [128]:
import pandas as pd
import json
import string

# Coordinate extraction for states

In [131]:
states = pd.read_csv("state_coordinates/csv/USA_notion.csv")
states["file_name"] = states["GeoJson File"].apply(lambda x: x.split("/")[-1])

In [139]:
def add_first_y_mirror(poly):
    points = [[point[0], -point[1]] for point in poly]
    first_point = points[0]
    points += [first_point]
    return points

In [140]:
def get_coordinates(geojson):
    polygon = geojson["features"][0]["geometry"]["coordinates"][0]
    points = add_first_y_mirror(polygon)
    return points

In [145]:
def get_problem_coordinates(geojson):
    coord = geojson["features"][0]["geometry"]["coordinates"]
    multi_poly = []
    for poly in coord:
        poly_points = add_first_y_mirror(poly[0])
        multi_poly.append(poly_points)
    return multi_poly

In [177]:
states_dict = {}
problem_state = {}
for f_name in states["file_name"].values.tolist():
    f_name_rep = f_name.replace("_", " ")
    with open(f"state_coordinates/geojson/{f_name_rep}") as f:
            geo_file = f.read()
    file_content = json.loads(geo_file)
    if len(file_content["features"][0]["geometry"]["coordinates"]) == 1 or f_name == "New_Jersey.geojson":
        print(f_name, "polygon")
        states_dict[f_name_rep] = ["polygon", get_coordinates(file_content)]
    else:
        print(f_name, "multipolygon")
        states_dict[f_name_rep] = ["multipolygon", get_problem_coordinates(file_content)]

alabama.geojson polygon
Alaska.geojson multipolygon
Arizona.geojson polygon
Arkansas.geojson polygon
California.geojson multipolygon
Colorado.geojson polygon
Connecticut.geojson polygon
Delaware.geojson polygon
Florida.geojson multipolygon
Georgia.geojson polygon
Hawaii.geojson multipolygon
Idaho.geojson polygon
Illinois.geojson polygon
Indiana.geojson polygon
Iowa.geojson polygon
Kansas.geojson polygon
Kentucky.geojson multipolygon
Louisiana.geojson polygon
Maine.geojson multipolygon
Maryland.geojson polygon
Massachusetts.geojson polygon
Michigan.geojson polygon
Minnesota.geojson polygon
Mississippi.geojson polygon
Missouri.geojson polygon
Montana.geojson polygon
Nebraska.geojson polygon
Nevada.geojson polygon
New_Hampshire.geojson polygon
New_Jersey.geojson polygon
New_Mexico.geojson polygon
New_York.geojson multipolygon
North_Carolina.geojson polygon
North_Dakota.geojson polygon
Ohio.geojson polygon
Oklahoma.geojson polygon
oregon.geojson polygon
Pennsylvania.geojson polygon
Rhode_I

In [178]:
coordinates = pd.DataFrame.from_dict(states_dict, orient="index", columns=["type", "coordinates"]).reset_index().rename({"index": "file_name"}, axis=1)
coordinates["state"] = coordinates["file_name"].apply(lambda x: string.capwords(x.split(".")[0]))

In [179]:
coordinates.head()

,file_name,type,coordinates,state
0,alabama.geojson,polygon,"[[-88.3956563, -30.3812365], [-88.3942367, -30...",Alabama
1,Alaska.geojson,multipolygon,"[[[177.9562368, -52.2352051], [177.9557082, -5...",Alaska
2,Arizona.geojson,polygon,"[[-114.5693264, -35.1720074], [-114.5693466, -...",Arizona
3,Arkansas.geojson,polygon,"[[-89.704387, -35.819607], [-89.7031009, -35.8...",Arkansas
4,California.geojson,multipolygon,"[[[-119.0693149, -33.4126665], [-119.0615902, ...",California


In [172]:
for state, c in coordinates[coordinates["type"] == "multipolygon"][["state", "coordinates"]].values:
   print(state)
   with open(f"state_coordinates/multipolygon/{state}.txt", 'w') as f:
       f.write(str(c))

Alaska
California
Florida
Hawaii
Kentucky
Maine
New York
Rhode Island


In [180]:
coordinates.to_csv("state_coordinates/csv/coordinates.csv", index=False)